# AY22 Artifical Intelligence (A) - Assignment 2022-2023
Chest X-ray Abnormalities Detection using Deep Learning: Development of a deep learning model to automatically localize and classify thoracic abnormalities from chest radiographs. (Option n°2)

© Copyright 2023, All rights reserved to Hans Haller, CSTE-CIDA Student at Cranfield Uni. SATM, Cranfield, UK.

https://www.github.com/Hnshlr

### GOOGLE COLAB OPTIONS (IGNORE THES STEPS IF NOT ON COLAB):

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp '/content/drive/MyDrive/CRANFIELD/9_AI/AI_ASSIGNMENT/AI_CODE_HALLER_388885/src.zip' '/content/'

In [ ]:
import os
!unzip /content/src.zip -d /content/

In [ ]:
os.chdir('/content/')

In [ ]:
!pip install pydicom

In [ ]:
# https://www.kaggle.com/ultralytics/yolov5
!cd /content/src/yolostuff && git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
!cd /content/src/yolostuff/yolov5 && pip install -r requirements.txt  # install dependencies
# cmd = "!python {yolo_dir}train.py --img 256 --batch 32 --epochs 2 --data {yaml_path} --weights {model} --cache"

### IMPORTS

In [ ]:
# IMPORTS=
import os
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import numpy as np
import cv2
import pydicom as dicom
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

# MY DATASET=
from src import ChestXrayDataset as CXD
from src.Preprocessing import *
from src.Postprocessing import *

### DATA PREPROCESSING

In [ ]:
# SETTINGS =
data_dir = "src/data/input/256x256/"                            # MAIN DIRECTORY CONTAINING THE DATA
train_df = pd.read_csv(data_dir + "train.csv")                  # TRAINING DATA
train_df_sizes = pd.read_csv(data_dir + "train_meta.csv")       # TRAINING DATA SIZES

# ADV. SETTINGS =
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.has_mps else "cpu")   # DO NOT TOUCH

# PREPROCESSING:
new_size = 256  # NEW SIZE OF THE IMAGES
train_df = preprocess_data(data_dir, train_df, train_df_sizes, new_size)  # DATA PREPROCESSING (+ SAVE -> train_clean.csv)
class_ids, class_names = class_ids_and_names(train_df)  # CLASS IDS AND NAMES

### FIRST APPROACH: RESNET18 FULL IMPLEMENTATION

In [ ]:
# TRANSFORMATION PIPELINE:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize the image
])

# DATASET:
dataset = CXD.ChestXrayDataset(csv_file="train_single.csv", data_dir=data_dir, transform=transform) # Create the dataset

# TRAIN/VALIDATION SPLIT:
ratio = 0.8
train_dataset, val_dataset = dataset.split(ratio)   # Split the dataset using the ratio

# DATALOADERS:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
print("Number of train images: ", len(train_dataset))
print("Number of validation images: ", len(val_dataset))
print("Number of batches: ", len(train_loader))
for i, (images, labels) in enumerate(train_loader):
    print("Images shape: ", images.shape)
    print("Labels shape: ", labels.shape)
    break

In [ ]:
# Define the model:
model = models.resnet18(pretrained=True)
num_classes = 15
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes) # Fully connected layer
model = model.to(device)

# Define the loss function and the optimizer:
criterion = nn.CrossEntropyLoss()   # Loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Optimizer

In [ ]:
# Start a timer:
import time
start_time = time.time()

# Train the model:
num_epochs = 30
for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    model.train()   # Set the model to training mode
    print("Epoch: ", epoch)
    for i, (images, labels) in enumerate(train_loader):
        # if i % 10 == 0:
            # print("Batch: "+str(i)+" began.")
        images = images.to(device)  # Move the images to the device
        labels = labels.to(device)  # Move the labels to the device
        optimizer.zero_grad()   # Zero the gradients
        outputs = model(images) # Forward pass
        loss = criterion(outputs, labels)   # Loss calculation
        loss.backward()     # Weight update
        optimizer.step()    # Gradient update
        train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs.data, 1)   # Get the predictions
        train_correct += torch.sum(preds == labels.data).sum().item()   # Calculate the number of correct predictions
        train_loss += loss.item() * images.size(0)  # Update the training loss
    train_acc = train_correct / len(train_dataset)  # Calculate the training accuracy
    train_loss = train_loss / len(train_dataset)    # Calculate the training loss
    print("Epoch: {}/{}...".format(epoch + 1, num_epochs),
          "Training Loss: {:.4f}...".format(train_loss),
          "Training Accuracy: {:.4f}".format(train_acc))

# Stop the timer:
end_time = time.time()
print("Resnet18 training took: ", np.round((end_time - start_time), 2), " seconds.")

In [ ]:
# Save the model:
# torch.save(model.state_dict(), "src/data/output/model.pth")

In [ ]:
# Load the model:
model.load_state_dict(torch.load("src/data/output/resnet18_e10.pth"))
# Then, re-test the model on the validation set.

In [ ]:
# Test the model on the validation set:
model.eval()
val_loss = 0.0  # Validation loss
val_correct = 0 # Number of correct predictions
for i, (images, labels) in enumerate(val_loader):
    images = images.to(device)  # Move the images to the device
    labels = labels.to(device)  # Move the labels to the device
    outputs = model(images)     # Forward pass
    loss = criterion(outputs, labels)   # Loss calculation
    val_loss += loss.item() * images.size(0)    # Update the validation loss
    _, preds = torch.max(outputs.data, 1)   # Get the predictions
    val_correct += torch.sum(preds == labels.data).sum().item()  # Calculate the number of correct predictions
val_acc = val_correct / len(val_dataset)    # Calculate the validation accuracy
val_loss = val_loss / len(val_dataset)  # Calculate the validation loss
print("Validation Loss: {:.4f}...".format(val_loss),
      "Validation Accuracy: {:.4f}".format(val_acc))

### NEW MODEL: YOLOv5 (CLONED FROM GITHUB)

In [ ]:
# https://www.kaggle.com/ultralytics/yolov5
# !git clone https://github.com/ultralytics/yolov5  # clone repo
# %cd yolov5
# !pip install -r requirements.txt  # install dependencies
# cmd = "!python {yolo_dir}train.py --img 256 --batch 32 --epochs 2 --data {yaml_path} --weights {model} --cache"

In [ ]:
# YOLO STUFF:
yolostuff_dir = "src/yolostuff/"
yolo_dir = yolostuff_dir + "yolov5/"
yaml_path = yolostuff_dir + "datasets/vinbigdata/vinbigdata.yaml"
model_path = yolostuff_dir + "yolov5/models/yolov5s.pt"

In [ ]:
ratio = 0.8
val_df = train_df.sample(frac=1-ratio, random_state=42)
train_df = train_df.drop(val_df.index)

In [ ]:
# Save all the images names in a .txt:
txt_file = ""
for row in train_df["image_id"]:
    txt_file += "./images/" + row + ".png\n"
txt_file_path = yolostuff_dir + "datasets/vinbigdata/train.txt"
txt_file_opened = open(txt_file_path, "w")
txt_file_opened.write(txt_file)
txt_file_opened.close()

# Save all the images names in a .txt:
txt_file = ""
for row in val_df["image_id"]:
    txt_file += "./images/" + row + ".png\n"
txt_file_path = yolostuff_dir + "datasets/vinbigdata/val.txt"
txt_file_opened = open(txt_file_path, "w")
txt_file_opened.write(txt_file)
txt_file_opened.close()

In [ ]:
!python {yolo_dir}train.py --img 256 --batch 32 --epochs 1 --data {yaml_path} --weights {model_path} --cache

In [ ]:
v5s_e30 = 'src/yolostuff/yolov5/runs/train/exp_v5s_e30/weights/best.pt'
v5x_e30 = 'src/yolostuff/yolov5/runs/train/exp_v5x_e30/weights/best.pt'
v5x_e60 = 'src/yolostuff/yolov5/runs/train/exp_v5x_e60/weights/best.pt'
model_path = v5x_e60

In [ ]:
!python {yolo_dir}detect.py --weights {model_path} --img 256 --conf 0.15 --iou 0.5 --source 'src/data/input/256x256/test' --save-txt --save-conf --exist-ok

### POST-PROCESSING OF THE YOLOv5 OUTPUTS:

In [ ]:
output_labels_path = "src/yolostuff/yolov5/runs/detect/exp/labels/"
test_meta_path = "src/data/input/256x256/test_meta.csv"
sample_submission_path = "src/data/input/sample_submission.csv"

In [ ]:
submission = postprocess_yolo(output_labels_path, test_meta_path, sample_submission_path)

## VISUALIZATION

In [ ]:
# Debug: Plot one image, its labels and its bounding boxes:
batch1 = next(iter(train_loader))

In [ ]:
index = 10
img = batch1[0][index].permute(1, 2, 0)
# In this format, the image is in RGB, but the values are between -1 and 1.
# We need to convert it to 0-255:
img = (img + 1) / 2
plt.imshow(img)
label = batch1[1][index].item()
bbox = batch1[2][index]
# Get first element of bbox, and convert it to a int:
print(bbox)
print(label)
# Debug: Plot the bounding boxes:
for i in range(0, len(bbox), 4):
    xmin = bbox[i].item()
    ymin = bbox[i+1].item()
    width = bbox[i+2].item()
    height = bbox[i+3].item()
    rect = plt.Rectangle((xmin, ymin), width, height, fill=False, color='red')
    plt.gca().add_patch(rect)
    plt.show()

In [ ]:
test_img_id = '013c169f9dad6f1f6485da961b9f7bf2'
test_img = cv2.imread("src/data/input/256x256/test/" + test_img_id + ".png")
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
plt.imshow(test_img)
test_img_predLabels = pd.read_csv("src/yolostuff/yolov5/runs/detect/exp/labels/" + test_img_id + ".txt", header=None)
# Keep the first row:
values = test_img_predLabels.iloc[0]
# Get the values:
values = values[0].split(" ")
x_mid_n = float(values[1])*256
y_mid_n = float(values[2])*256
width_n = float(values[3])*256
height_n = float(values[4])*256
x_min_n = x_mid_n - width_n / 2
y_min_n = y_mid_n - height_n / 2
x_max_n = x_mid_n + width_n / 2
y_max_n = y_mid_n + height_n / 2
# Plot a rectangle above the image (dont resize):
plt.gca().add_patch(plt.Rectangle((x_min_n, y_min_n), width_n, height_n, fill=False, color='red'))
plt.show()

### Acknowledgements

This code was developed as part of the Artificial Intelligence course at Cranfield University, UK.